## CONNECT TO SDE DATABASE

- This notebook pushes the simulated BMSB data to a Postgres database
- Utilizing an .sde database, Github, and Google Cloud Run, this data can be displayed in a ArcGIS Online Web Map

In [125]:
# Library for data manipulation and analysis
import pandas as pd

# Library for working with ArcGIS tools and functionalities
import arcpy
import arcgis

# Libraries for working with PostgreSQL
import psycopg2
from psycopg2 import sql

# Library for handling JSON data
import json

# Library for making HTTP requests and interacting with web services
import requests

# Library for interacting with the operating system, managing file paths, and executing system commands
import os

# Library for handling warnings generated during code execution
import warnings

# Library for displaying images in Jupyter Notebooks
from IPython.display import Image

# Library for generating random numbers
import random

# Library for handling zip files and working with IO streams
import zipfile
import io

# Library for working with dates
from datetime import date

import urllib.request

In [143]:
#path to local database
file_gdb = r"C:\Users\15612\Documents\Arc_II\Final_Project\BMSB_Project_2\BMSB_Project_2.gdb"

# Establish SDE Connection via PGAdmin & Catalog Pane in ArcGIS Pro
sde = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\4. Project\ArcPro Files\ArcII_Final Project\PostgreSQL-35-bmsb_project_shared(postgres).sde'

In [146]:
# Database connection parameters
dbname = 'PostgreSQL-35-bmsb_project_shared(postgres).sde'
platform = 'POSTGRESQL'
user = 'postgres'
password = 'Hyderabad43%'
instance = '35.238.64.215'
port = '5432'
auth = 'DATABASE_AUTH'
save = 'SAVE_USERNAME'
db = 'gis5572'

# Create database connection using the provided parameters
arcpy.management.CreateDatabaseConnection(
    out_folder_path=r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\4. Project\ArcPro Files\ArcII_Final Project',
    out_name=dbname,
    database_platform=platform,
    instance=instance,
    account_authentication=auth,
    username=user,
    password=password,
    save_user_pass=save,
    database=db
)

<Result 'F:\\1. UMN MGIS\\1. Semesters\\4th Semester\\1. ArcGIS II\\4. Project\\ArcPro Files\\ArcII_Final Project\\PostgreSQL-35-bmsb_project_shared(postgres).sde'>

### Export from local FGDB to PostgresSQL

In [149]:
# Function to upload feature class to SDE
def upload_feature_class_to_sde(input_feature_class, output_sde_connection, output_sde_feature_class):
    try:
        arcpy.FeatureClassToFeatureClass_conversion(input_feature_class, output_sde_connection, output_sde_feature_class)
        print(f"Feature class '{output_sde_feature_class}' uploaded successfully to SDE.")
    # function uses a try-except block to handle potential errors during the feature class conversion
    except Exception as e:
        print(f"Error uploading feature class to SDE: {str(e)}")

# Paths and parameters
input_feature_class_1 = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\4. Project\ArcPro Files\ArcII_Final Project\ArcII_Final Project.gdb\MN_Cities_15_ranked'
input_feature_class_2 = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\4. Project\ArcPro Files\ArcII_Final Project\ArcII_Final Project.gdb\MN_Cities_175__ranked'
input_feature_class_3 = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\4. Project\ArcPro Files\ArcII_Final Project\ArcII_Final Project.gdb\MN_Cities_2_ranked'
output_sde_connection = r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\4. Project\ArcPro Files\ArcII_Final Project\PostgreSQL-35-bmsb_project_shared(postgres).sde'

# Upload feature classes to SDE
upload_feature_class_to_sde(input_feature_class_1, output_sde_connection, 'MN_Cities_15_ranked')
upload_feature_class_to_sde(input_feature_class_2, output_sde_connection, 'MN_Cities_175_ranked')
upload_feature_class_to_sde(input_feature_class_3, output_sde_connection, 'MN_Cities_2_ranked')

Feature class 'MN_Cities_15_ranked' uploaded successfully to SDE.
Feature class 'MN_Cities_175_ranked' uploaded successfully to SDE.
Feature class 'MN_Cities_2_ranked' uploaded successfully to SDE.


In [134]:
# Define a spatial reference with ID 4326 (WGS 84)
spatial_ref = arcpy.SpatialReference(4326)

In [135]:
def project(in_features):

    arcpy.management.Project(
        in_dataset = in_features,
        out_dataset = os.path.join(file_gdb,in_features + '_WGS84'),
        out_coor_system = spatial_ref
    )

In [ ]:
project('MN_Cities_15_ranked')
project('MN_Cities_175_ranked')
project('MN_Cities_2_ranked')

In [29]:
BMSB_rank_list = [os.path.join(file_gdb,'MN_Cities_15_ranked_WGS84'),os.path.join(file_gdb,'MN_Cities_175_ranked_WGS84'),os.path.join(file_gdb,'MN_Cities_2_ranked_WGS84')]

# Set variable to SDE database
sde_gdb = os.path.join(file_path,dbname)

# Add data to sde database
for BMSB_rank in BMSB_rank_list:

    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features = BMSB_rank,
        Output_Geodatabase = sde_gdb
    )